In [3]:
import os
import torch
import torchvision
import wandb
from torchvision import transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [4]:
def get_transforms(augment):
    base = [
        transforms.Resize((299, 299)),  # Keep original size for compatibility
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
    if augment == 'yes':
        aug = [
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(20),
            transforms.ColorJitter(0.2, 0.2, 0.2, 0.2)
        ]
        return transforms.Compose(aug + base)
    else:
        return transforms.Compose(base)

In [5]:
def get_dataloaders(batch_size, augment):
    transform = get_transforms(augment)
    dataset = ImageFolder('/kaggle/input/nature-12k/inaturalist_12K/train', transform=transform)
    val_pct = 0.2

    # Stratified split
    targets = np.array(dataset.targets)
    indices = np.arange(len(targets))
    train_indices, val_indices = [], []
    for c in np.unique(targets):
        class_indices = indices[targets == c]
        np.random.shuffle(class_indices)
        split = int(len(class_indices) * (1 - val_pct))
        train_indices.extend(class_indices[:split])
        val_indices.extend(class_indices[split:])

    train_set = Subset(dataset, train_indices)
    val_set = Subset(dataset, val_indices)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
    return train_loader, val_loader

In [6]:
def build_model(dense_layers, dropout, layers_unfreeze):
    # Load ResNet50
    model = torchvision.models.resnet50(pretrained=True)
    
    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze last N layers if specified
    if layers_unfreeze is not None:
        total_layers = list(model.children())
        for layer in total_layers[-layers_unfreeze:]:
            for param in layer.parameters():
                param.requires_grad = True

    # Replace final fully connected layer
    in_features = model.fc.in_features
    layers = []
    for units in dense_layers:
        layers.append(nn.Linear(in_features, units))
        layers.append(nn.ReLU())
        if dropout > 0:
            layers.append(nn.Dropout(dropout))
        in_features = units
    layers.append(nn.Linear(in_features, 10))
    model.fc = nn.Sequential(*layers)
    return model

In [7]:
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    total, correct, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        loss_sum += loss.item() * x.size(0)
        preds = torch.argmax(outputs, 1)
        correct += (preds == y).sum().item()
        total += x.size(0)
    return loss_sum / total, correct / total

In [8]:
def validate(model, loader, device):
    model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            loss_sum += loss.item() * x.size(0)
            preds = torch.argmax(outputs, 1)
            correct += (preds == y).sum().item()
            total += x.size(0)
    return loss_sum / total, correct / total

In [9]:
def sweep_train():
    wandb.init(project='DA6401_A2')
    config = wandb.config
    run_name = (
        f"bs{config.batch_size}_"
        f"dense{config.dense_layers}_"
        f"drop{config.dropout}_"
        f"unfrz{config.layers_unfreeze}_"
        f"aug{config.augment_data}_"
        f"ep{config.epochs}"
    )
    wandb.run.name = run_name

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, val_loader = get_dataloaders(config.batch_size, config.augment_data)
    model = build_model(config.dense_layers, config.dropout, config.layers_unfreeze)
    model.to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    best_val_acc = 0.0
    for epoch in range(config.epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, device)
        val_loss, val_acc = validate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{config.epochs} | "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        wandb.log({
            'epoch': epoch+1,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_accuracy': val_acc
        })
        if val_acc > best_val_acc:
            best_val_acc = val_acc
    wandb.log({'best_val_accuracy': best_val_acc})

In [10]:
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'dense_layers': {'values': [[256], [128], [64]]},
        'layers_unfreeze': {'values': [None, 20, 30]},
        'dropout': {'values': [0, 0.2, 0.3]},
        'augment_data': {'values': ['yes', 'no']},
        'batch_size': {'values': [128, 64, 256]},
        'epochs': {'values': [5, 10]}
    }
}

In [1]:
wandb.login(key="49f8f505158ee3693f0cacf0a82118bd4e636e8c")
sweep_id = wandb.sweep(sweep_config, project='DA6401_A2')
wandb.agent(sweep_id, function=sweep_train)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: surendarmohan283 (surendarmohan283-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: 9om4xby1
Sweep URL: https://wandb.ai/surendarmohan283-indian-institute-of-technology-madras/DA6401_A2/sweeps/9om4xby1


wandb: Agent Starting Run: pqnsa05u with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 64
wandb: 	dense_layers: [128]
wandb: 	dropout: 0
wandb: 	epochs: 5
wandb: 	layers_unfreeze: None


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s] 


...

best_val_accuracy,▁
epoch,▁▃▅▆█
train_acc,▁▄▅▇█
train_loss,█▅▄▂▁
val_accuracy,▁▃▅██
val_loss,▄█▅▁▇
best_val_accuracy,0.4465
epoch,5
train_acc,0.59682
train_loss,1.17999
val_accuracy,0.441


wandb: Agent Starting Run: o9khm5sf with config:
wandb: 	augment_data: yes
wandb: 	batch_size: 64
wandb: 	dense_layers: [256]
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	layers_unfreeze: 30


Epoch 1/10 | Train Loss: 2.1497 | Train Acc: 0.2225 | Val Loss: 2.2888 | Val Acc: 0.2405
Epoch 2/10 | Train Loss: 1.9727 | Train Acc: 0.2962 | Val Loss: 2.0127 | Val Acc: 0.2925
Epoch 3/10 | Train Loss: 1.8525 | Train Acc: 0.3462 | Val Loss: 2.1568 | Val Acc: 0.2870
Epoch 4/10 | Train Loss: 1.8008 | Train Acc: 0.3647 | Val Loss: 1.7346 | Val Acc: 0.3980
Epoch 5/10 | Train Loss: 1.7257 | Train Acc: 0.3970 | Val Loss: 1.8172 | Val Acc: 0.3885
Epoch 6/10 | Train Loss: 1.6593 | Train Acc: 0.4159 | Val Loss: 1.7451 | Val Acc: 0.3745


wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▂▄▅▇█
train_acc,▁▄▅▆▇█
train_loss,█▅▄▃▂▁
val_accuracy,▁▃▃██▇
val_loss,█▅▆▁▂▁
epoch,6
train_acc,0.41593
train_loss,1.65935
val_accuracy,0.3745
val_loss,1.74507


In [12]:
def get_test_dataloader(batch_size):
    """Create dataloader for test data"""
    transform = get_transforms(augment='no')  # No augmentation for test
    test_dataset = ImageFolder('/kaggle/input/nature-12k/inaturalist_12K/val', transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    return test_loader

def evaluate_test_accuracy():
    """Evaluate the best model on test set"""
    # Best configuration
    config = {
        'batch_size': 256,
        'dense_layers': [128],
        'dropout': 0.2,
        'layers_unfreeze': 20,
        'augment_data': 'no',
        'epochs': 5
    }
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Get dataloaders
    train_loader, val_loader = get_dataloaders(config['batch_size'], config['augment_data'])
    test_loader = get_test_dataloader(config['batch_size'])
    
    # Build model
    model = build_model(config['dense_layers'], config['dropout'], config['layers_unfreeze'])
    model.to(device)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    
    # Evaluate on test set
    test_loss, test_acc = validate(model, test_loader, device)
    print("\nFinal Test Results:")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")

if __name__ == '__main__':
    evaluate_test_accuracy()

Final Test Results:
Test Loss: 0.5480 | Test Accuracy: 0.8298
